In [1]:
#get working directory
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import os
os.getcwd()

'C:\\Users\\Zain Ul Abideen\\Dropbox\\Code4Pol'

In [2]:
airport_lookupDF = pd.read_csv('https://s3.amazonaws.com/jsw.dsprojects/AirlinePredictions/Airport_Lookup.csv', 
                                header = 0) # Airport codes

In [3]:
trainDF = pd.read_csv('FinalFlightsNumeric.csv', header = 0)

In [4]:
print (trainDF.shape)
trainDF.head()

(3752717, 31)


,Unnamed: 0,DATE,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,TAIL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,...,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,HDAYS,ARR_HOUR,DEP_HOUR,CARRIER_NAME,ORIGIN_AIRPORT,DESTINATION_AIRPORT,CARRIER_CODE
0,1,2017-01-01,2017,1,1,7,AA,N3CGAA,11292,14107,...,NaN,NaN,NaN,1,13,11,American Airlines Inc.,"Denver, CO: Denver International","Phoenix, AZ: Phoenix Sky Harbor International",1
1,2,2017-01-01,2017,1,1,7,AA,N3CGAA,14107,14057,...,NaN,NaN,NaN,1,17,15,American Airlines Inc.,"Phoenix, AZ: Phoenix Sky Harbor International","Portland, OR: Portland International",1
2,3,2017-01-01,2017,1,1,7,AA,N3EVAA,11278,13303,...,NaN,NaN,NaN,1,9,6,American Airlines Inc.,"Washington, DC: Ronald Reagan Washington National","Miami, FL: Miami International",1
3,4,2017-01-01,2017,1,1,7,AA,N3NAAA,13303,12953,...,NaN,NaN,NaN,1,17,14,American Airlines Inc.,"Miami, FL: Miami International","New York, NY: LaGuardia",1
4,5,2017-01-01,2017,1,1,7,AA,N3FCAA,14107,11292,...,12.0,0.0,16.0,1,0,22,American Airlines Inc.,"Phoenix, AZ: Phoenix Sky Harbor International","Denver, CO: Denver International",1


In [5]:
trainDF.dtypes

Unnamed: 0               int64
DATE                    object
YEAR                     int64
MONTH                    int64
DAY_OF_MONTH             int64
DAY_OF_WEEK              int64
UNIQUE_CARRIER          object
TAIL_NUM                object
ORIGIN_AIRPORT_ID        int64
DEST_AIRPORT_ID          int64
CRS_DEP_TIME             int64
DEP_DELAY              float64
CRS_ARR_TIME             int64
ARR_DELAY              float64
CANCELLED                int64
CANCELLATION_CODE       object
DIVERTED                 int64
CRS_ELAPSED_TIME       float64
DISTANCE                 int64
CARRIER_DELAY          float64
WEATHER_DELAY          float64
NAS_DELAY              float64
SECURITY_DELAY         float64
LATE_AIRCRAFT_DELAY    float64
HDAYS                    int64
ARR_HOUR                 int64
DEP_HOUR                 int64
CARRIER_NAME            object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
CARRIER_CODE             int64
dtype: object

In [6]:
#trainDF.loc[trainDF['ARR_DELAY'].isnull()][['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY']]

In [7]:
trainDF = trainDF[trainDF['ARR_DELAY'].notnull()]
trainDF.shape

(3690440, 31)

In [8]:
#drop the unnamed column
trainDF.drop(trainDF.columns[0], axis = 1, inplace = True)

In [9]:
#create a lookup table
carrierDF = trainDF[['UNIQUE_CARRIER', 'CARRIER_CODE']].drop_duplicates() # Only get unique examples

In [10]:
carrierDF

,UNIQUE_CARRIER,CARRIER_CODE
0,AA,1
460,AS,2
687,B6,3
794,EV,5
1277,DL,4
2668,F9,6
2872,NK,8
3056,OO,9
3321,WN,12
5665,HA,7


In [11]:
trainDF.drop('UNIQUE_CARRIER', axis = 1, inplace = True)

In [12]:
#creaeting a holiday table
hdaysDF = trainDF[['MONTH', 'DAY_OF_MONTH', 'HDAYS']].drop_duplicates()

In [13]:
#separating variables

scalingDF = trainDF[['DISTANCE', 'HDAYS']].astype('float') # Numerical features
categDF = trainDF[['MONTH', 'DAY_OF_MONTH', 'ORIGIN_AIRPORT_ID', 
                    'DEST_AIRPORT_ID', 'ARR_HOUR', 'DEP_HOUR', 
                    'CARRIER_CODE', 'DAY_OF_WEEK']] # Categorical features

In [14]:
from sklearn.preprocessing import OneHotEncoder
    
encoder = OneHotEncoder() # Create encoder object
categDF_encoded = encoder.fit_transform(categDF) # Can't convert this to dense array: too large!

In [15]:
type(categDF_encoded)

scipy.sparse.csr.csr_matrix

In [16]:
from scipy import sparse # Need this to create a sparse array
scalingDF_sparse = sparse.csr_matrix(scalingDF)

In [17]:
x_final = sparse.hstack((scalingDF_sparse, categDF_encoded))

In [18]:
y_final = trainDF['ARR_DELAY'].values

In [19]:
#splitting the data
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final,y_final,test_size = 0.2,random_state = 0) # Do 80/20 split

C:\Users\Zain Ul Abideen\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [20]:
# getting the numerical variabels to scale them
x_train_numerical = x_train[:, 0:2].toarray() # We only want the first two features which are the numerical ones.
x_test_numerical = x_test[:, 0:2].toarray()

In [21]:
#standardizing the data 
from sklearn.preprocessing import StandardScaler
    
scaler = StandardScaler() # create scaler object
scaler.fit(x_train_numerical) # fit with the training data ONLY
x_train_numerical = sparse.csr_matrix(scaler.transform(x_train_numerical)) # Transform the data and convert to sparse
x_test_numerical = sparse.csr_matrix(scaler.transform(x_test_numerical))

In [22]:
#replacing the numerical variables
x_train[:, 0:2] = x_train_numerical
x_test[:, 0:2] = x_test_numerical

In [23]:

from sklearn.linear_model import SGDRegressor
from sklearn.grid_search import GridSearchCV
import numpy as np
    
SGD_params = {'alpha': 10.0**-np.arange(1,7)} # Suggested range we try
SGD_model = GridSearchCV(SGDRegressor(random_state = 0), SGD_params, scoring = 'mean_absolute_error', cv = 5) # Use 5-fold CV 
SGD_model.fit(x_train, y_train) # Fit the model

C:\Users\Zain Ul Abideen\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\Zain Ul Abideen\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\Zain Ul Abideen\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\Zain Ul Abideen\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.

GridSearchCV(cv=5, error_score='raise',
       estimator=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=0, shuffle=True, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([  1.00000e-01,   1.00000e-02,   1.00000e-03,   1.00000e-04,
         1.00000e-05,   1.00000e-06])},
       pre_dispatch='2*n_jobs', refit=True, scoring='mean_absolute_error',
       verbose=0)

In [24]:
from sklearn.metrics import mean_absolute_error
    
y_true, y_pred = y_test, SGD_model.predict(x_test) # Predict on our test set
    
print ('Mean absolute error of SGD regression was:')
print(mean_absolute_error(y_true, y_pred))

Mean absolute error of SGD regression was:
21.4610260668


In [25]:
def max_num_flights(codes):
    '''
    This is a function for the delay prediction function to use for calculating the number of 
    flights in the database for a given city.
        
    Inputs: list of codes retrived in the delay_prediction function
    Output: The code with the largest number of flights.
    '''
    num_store = np.zeros(len(codes)) # Store all possible airport codes
    
    if len(codes) < 1:
        print('Try entering your city/airport again. No matching airports found.') # In case the aiport didn't show up 
        return                                                                     # in the data
    
    for i in range(len(codes)):
        num_flights = trainDF.MONTH[trainDF.ORIGIN_AIRPORT_ID == codes[i]].count() # Count number of rows in data for each
        num_store[i] = num_flights                                                 # airport code
           
        
    # Now find the maximum row
        
    max_ind = int(np.where(num_store == max(num_store))[0])
    # Now we know which code had the most flights. Return it.
    return(codes[max_ind])

In [103]:
def delay_prediction(origin = 'Fort Worth', destination = 'Chicago', carrier = 'American', 
                    dept_time = 17, arr_time = 19, month = 5, day = 15, weekday = 'Wednesday'):
                        
    '''
    This function allows you to input all of your flight information (no leaks!) and
    the function will return how late your flight will arrive based on the output from the 
    SGD Regressor.
        
    Inputs: 
        
            Origin (enter this as a city, state combo, or include the airport name (such as Bush
                        or Hobby). This will automatically calculate which airport you meant.
                 
            Destination (same as Origin, entered as a string)
                 
            Carrier (which Airline, use a string to represent the name (such as 'American' or 'United')
                 
            Departing Hour scheduled (just the hour of departure, based on a 24 hour cycle. This means
                                noon would be 12, and midnight would be 0.)
                
            Arriving Hour scheduled (same format as departing)
                 
            Month (the month the flight is scheduled for)
                 
            Day (the day of the month the flight is scheduled for)
                 
            Weekday (Enter as a capitalized word, such as 'Monday')
                 
            Available Carriers:
            AirTran
            Alaska
            American
            Delta
            Endeavor
            Envoy
            ExpressJet
            Frontier
            Hawaiian
            JetBlue
            Mesa
            SkyWest
            Southwest
            United
            US Airways
            Virgin
            
    Outputs: Estimated delay for the arrival (in minutes, can be negative if the flight is expected to arrive early)
    '''
         
    # Create a dict for our Airlines. Based on the carrierDF.
         
    carrier_dict = {'Endeavor':1, 'American':2, 'Alaska':3, 'JetBlue':4, 'Delta':5,
                    'ExpressJet':6, 'Frontier':7, 'AirTran':8, 'Hawaiian':9, 'Envoy':10,
                    'SkyWest':11, 'United':12, 'US Airways':13, 'Virgin':14,
                    'Southwest':15, 'Mesa':16}
                         
    # Another for day of the week
         
    weekday_dict = {'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4,
                    'Friday':5, 'Saturday':6, 'Sunday':7}
                         
    # Now find the corresponding airport codes for our origin and destination. 
         
    origin_codes = list(airport_lookupDF[airport_lookupDF.Description.str.contains(origin)].Code)
    destination_codes = list(airport_lookupDF[airport_lookupDF.Description.str.contains(destination)].Code)
         
    # From these codes found in the lookup table, see which one had the largest number of flights.
                                                                                                                                                                                                                                                                                                                                         
    origin_code = max_num_flights(origin_codes)
    destination_code = max_num_flights(destination_codes)
         
    # Now that we have these codes, we can look up the other parameters necessary.
         
    # Start with looking up HDAYS.
         
    hdays = np.array(int(hdaysDF[(hdaysDF.MONTH == month) & (hdaysDF.DAY_OF_MONTH == day)].HDAYS))
         
    # Now find the distance between the two airports. Add a try/except to catch any problems.
       
    try:
        distance = np.array(float(trainDF[(trainDF.ORIGIN_AIRPORT_ID == origin_code) & 
                    (trainDF.DEST_AIRPORT_ID == destination_code)].DISTANCE.drop_duplicates()))
    except:
        print ('Route was not found in the data. Please try a different nearby city or a new route.')
        return
         
    carrier_num = carrier_dict[carrier]
    weekday_num = weekday_dict[weekday]
         
    # Now that we have all of our values, we can start combining them together.
         
    # First, create our numeric array of distance and hdays.
         
    numerical_values = np.c_[distance, hdays]
         
    # Scale the features
         
    numerical_values_scaled = scaler.transform(numerical_values)
        
    # Now create our array of categorical values.
         
    categorical_values = np.zeros(8)
    categorical_values[0] = int(month)
    categorical_values[1] = int(day)
    categorical_values[2] = int(origin_code)
    categorical_values[3] = int(destination_code)
    categorical_values[4] = int(arr_time)
    categorical_values[5] = int(dept_time)
    categorical_values[6] = int(carrier_num)
    categorical_values[7] = int(weekday_num)
         
    # Apply the one-hot encoding to these.
         
    categorical_values_encoded = encoder.transform([categorical_values]).toarray()
         
    # Combine these into the final test example that goes into the model
         
    final_test_example = np.c_[numerical_values_scaled, categorical_values_encoded]
         
    # Now predict this with the model 
         
    pred_delay = SGD_model.predict(final_test_example)
    #print ('Your predicted delay is', int(pred_delay[0]), 'minutes.')
    delay_time = int(pred_delay[0])
    return delay_time # End of function

In [104]:
delay_prediction(origin = 'Washington, DC', destination = 'Seattle', 
                carrier = 'United', dept_time = 17, arr_time = 21,
                month = 3, day = 22, weekday = 'Monday')

9

In [105]:
delay = delay_prediction(origin = 'Boston', destination = 'Houston', 
                carrier = 'AirTran', dept_time = 22, arr_time = 1,
                month = 3, day = 19, weekday = 'Thursday')

deptime_df = trainDF[['CRS_DEP_TIME']].drop_duplicates()

deptime_df

In [153]:
dep_time = list(range(24))
arr_time = list(range(24))

delay_time = []

dest = ['Houston', 'Chicago', 'Washington, DC', 'Miami', 'San Francisco', 'New York']
carr = ['AirTran', 'Alaska', 'American', 'Delta', 'Endeavor', 'Envoy', 'ExpressJet', 'Frontier', 'Hawaiian', 'JetBlue',
        'SkyWest', 'United']

delayss = []
for d in dest:
    for c in carr:
        delay = delay_prediction(origin = 'Boston', destination = d, 
                carrier = c, dept_time = 17, arr_time = 21,
                month = 3, day = 19, weekday = 'Thursday')
        print (c)
        print (d)
        print (delay)
        delayss.append ({'dest': d, 'delay': delay, 'carrier': c })

AirTran
Houston
8
Alaska
Houston
12
American
Houston
6
Delta
Houston
11
Endeavor
Houston
7
Envoy
Houston
4
ExpressJet
Houston
8
Frontier
Houston
8
Hawaiian
Houston
10
JetBlue
Houston
2
SkyWest
Houston
10
United
Houston
8
AirTran
Chicago
13
Alaska
Chicago
16
American
Chicago
10
Delta
Chicago
15
Endeavor
Chicago
11
Envoy
Chicago
8
ExpressJet
Chicago
13
Frontier
Chicago
12
Hawaiian
Chicago
14
JetBlue
Chicago
7
SkyWest
Chicago
14
United
Chicago
13
AirTran
Washington, DC
11
Alaska
Washington, DC
15
American
Washington, DC
9
Delta
Washington, DC
14
Endeavor
Washington, DC
10
Envoy
Washington, DC
7
ExpressJet
Washington, DC
11
Frontier
Washington, DC
11
Hawaiian
Washington, DC
13
JetBlue
Washington, DC
6
SkyWest
Washington, DC
13
United
Washington, DC
12
AirTran
Miami
11
Alaska
Miami
15
American
Miami
9
Delta
Miami
14
Endeavor
Miami
10
Envoy
Miami
7
ExpressJet
Miami
11
Frontier
Miami
11
Hawaiian
Miami
13
JetBlue
Miami
5
SkyWest
Miami
13
United
Miami
12
AirTran
San Francisco
17
Alaska
San Fran

In [154]:
delayss

[{'carrier': 'AirTran', 'delay': 8, 'dest': 'Houston'},
 {'carrier': 'Alaska', 'delay': 12, 'dest': 'Houston'},
 {'carrier': 'American', 'delay': 6, 'dest': 'Houston'},
 {'carrier': 'Delta', 'delay': 11, 'dest': 'Houston'},
 {'carrier': 'Endeavor', 'delay': 7, 'dest': 'Houston'},
 {'carrier': 'Envoy', 'delay': 4, 'dest': 'Houston'},
 {'carrier': 'ExpressJet', 'delay': 8, 'dest': 'Houston'},
 {'carrier': 'Frontier', 'delay': 8, 'dest': 'Houston'},
 {'carrier': 'Hawaiian', 'delay': 10, 'dest': 'Houston'},
 {'carrier': 'JetBlue', 'delay': 2, 'dest': 'Houston'},
 {'carrier': 'SkyWest', 'delay': 10, 'dest': 'Houston'},
 {'carrier': 'United', 'delay': 8, 'dest': 'Houston'},
 {'carrier': 'AirTran', 'delay': 13, 'dest': 'Chicago'},
 {'carrier': 'Alaska', 'delay': 16, 'dest': 'Chicago'},
 {'carrier': 'American', 'delay': 10, 'dest': 'Chicago'},
 {'carrier': 'Delta', 'delay': 15, 'dest': 'Chicago'},
 {'carrier': 'Endeavor', 'delay': 11, 'dest': 'Chicago'},
 {'carrier': 'Envoy', 'delay': 8, 'dest

In [155]:
df_delays =pd.DataFrame(delayss)

In [157]:
df_delays.to_csv('delays.csv', sep=',')